In [3]:
import pandas as pd
import timeit

In [8]:
def json_to_csv(data):
    # Open a file in write mode.
    with open('teste_json_direto.csv', 'w') as f:
        # Write all the dictionary keys in a file with commas separated.
        f.write(','.join(data[0].keys()))
        f.write('\n') # Add a new line
        for row in data:
            # Write the values in a row.
            f.write(','.join(str(x) for x in row.values()))
            f.write('\n') # Add a new line

In [9]:
def json_pd_to_csv(data):
    df = pd.read_json(data)
    df.to_csv('teste_dataframe.csv')

In [10]:
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

SETTINGS = {
  "save_to_csv": True
}

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'credentials.json'
def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()

  return response

if __name__ == '__main__':
  # Disable OAuthlib's HTTPs verification when running locally.
  # *DO NOT* leave this option enabled when running in production.
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

  youtubeAnalytics = get_service()
  response_dict = execute_api_request(
      youtubeAnalytics.reports().query,
      ids='channel==MINE',
      startDate='2023-05-01',
      endDate='2023-05-19',
      metrics='estimatedMinutesWatched,views,likes,subscribersGained',
      dimensions='day',
      sort='day'
  )

  start = timeit.default_timer()
  json_to_csv(response_dict)
  print(f'Diferença de tempo 1: {timeit.default_timer() - start}')

  start = timeit.default_timer()
  json_pd_to_csv(response_dict)
  print(f'Diferença de tempo 2: {timeit.default_timer() - start}')

  

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=564244369041-9k5heah4o92ea1g5brqm1de5sks3je92.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=aB9cjhuZOClOCXFk7tfTVKJu1C7eGz&prompt=consent&access_type=offline


InvalidGrantError: (invalid_grant) Bad Request